In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn.metrics

In [13]:
nround     = 200
repeat     = 10
operator   = 4
newfeature = 64
numcv      = 10
param = {'bst:eta':0.02, 
         'silent' : 1,
         'objective':'binary:logistic',
         'bst:max_depth':11,
         'base_score':0.5,
         'subsample':0.9,
         'bst:colsample_bytree':0.5,
         'scale_pos_weight':1,
         'min_child_weight':5}
plst = param.items()
plst +=  [('eval_metric', 'auc')]
head = []
for i in range(64):
    head.append('X' + str(i+1))
head.append('Y')
Data = []
for i in range(5):
    temp = pd.read_csv('CSV/' + str(i+1) + 'year.csv', names = head, header = None, na_values = '?')
    temp = temp.fillna(temp.median())
    Data.append(temp)

In [14]:
for i in range(5):
    
    data   = Data[i]
    datay  = data['Y'].copy()
    datax  = data.drop(['Y'], axis = 1)
    p      = np.zeros((datax.shape[1]),dtype = float) + 1/float(datax.shape[1])
    result = pd.DataFrame(index = range(repeat), columns = range(numcv+1))
    
    for k in range(repeat):
        
        for f in range(newfeature):
                
            o  = np.random.randint(operator, size = 1);
            f1 = np.random.multinomial(1, p, size = 1)[0]
            f2 = np.random.multinomial(1, p, size = 1)[0]
                
            if(o == 0):
                c = '(' + datax.columns[f1 == 1][0] + ')+(' + datax.columns[f2 == 1][0] + ')'
                if c not in datax.columns:
                    datax[c] = datax[datax.columns[f1 == 1][0]] + datax[datax.columns[f2 == 1][0]]
                    p        = np.append(p,[0], axis = 0)
            elif(o == 1):
                c = '(' + datax.columns[f1 == 1][0] + ')-(' + datax.columns[f2 == 1][0] + ')'
                if c not in datax.columns:
                    datax[c] = datax[datax.columns[f1 == 1][0]] - datax[datax.columns[f2 == 1][0]]
                    p        = np.append(p,[0], axis = 0)
            elif(o == 2):
                c = '(' + datax.columns[f1 == 1][0] + ')*(' + datax.columns[f2 == 1][0] + ')'
                if c not in datax.columns:
                    datax[c] = datax[datax.columns[f1 == 1][0]] * datax[datax.columns[f2 == 1][0]]
                    p        = np.append(p,[0], axis = 0)
            else:
                c = '(' + datax.columns[f1 == 1][0] + ')/(' + datax.columns[f2 == 1][0] + ')'
                if c not in datax.columns:
                    datax[c] = datax[datax.columns[f1 == 1][0]] / datax[datax.columns[f2 == 1][0]]
                    p        = np.append(p,[0], axis = 0)
        
        cv = np.random.randint(numcv, size = data.shape[0])
        fscore = {}
        
        for j in range(numcv):
            
            trainx = datax[cv != j]
            testx  = datax[cv == j]
            trainy = datay[cv != j]
            testy  = datay[cv == j]
            train  = xgb.DMatrix(trainx, label = trainy)
            test   = xgb.DMatrix(testx, label = testy)
            
            evallist = [(test,'test'), (train,'train')]       
            bst      = xgb.train(plst, train, nround, evallist, early_stopping_rounds = 100)
            result.loc[k, j] = float(bst.eval(test,'test', bst.best_iteration).split(":")[1])
            fs   = bst.get_fscore()
            for key in fs.keys():
                if key in fscore.keys():
                    fscore[key] += fs[key]
                else:
                    fscore[key] = fs[key]
            
        features = {}
        for key in fscore.keys():
            if float(fscore[key]) >= np.percentile(fscore.values(), 10):
                features[key] = float(fscore[key])   
        keys = features.keys()
        p = np.zeros((len(keys)),dtype = float)
        datanew = pd.DataFrame()
        for m in range(len(keys)):
            key = keys[m]
            newdata = datax[key].copy()
            datanew = pd.concat([datanew, newdata], axis = 1)
            p[m] = float(fscore[key])        
        datax = datanew
        p = p/np.sum(p)

        features = pd.Series(p, index = keys)
        features = features.sort_values(ascending = False)
        features.to_csv('R/' + str(i+1) + 'year' + str(k+1) + 'repeat.csv')
        
    result[numcv] = result.mean(1)
    result.to_csv('R/' + str(i+1) + 'yearAuc.csv')

[0]	test-auc:0.724542	train-auc:0.764776
Multiple eval metrics have been passed: 'train-auc' will be used for early stopping.

Will train until train-auc hasn't improved in 100 rounds.
[1]	test-auc:0.827651	train-auc:0.888012
[2]	test-auc:0.916587	train-auc:0.94496
[3]	test-auc:0.919268	train-auc:0.951745
[4]	test-auc:0.945413	train-auc:0.966993
[5]	test-auc:0.947105	train-auc:0.973075
[6]	test-auc:0.951701	train-auc:0.980693
[7]	test-auc:0.945349	train-auc:0.984076
[8]	test-auc:0.944455	train-auc:0.987273
[9]	test-auc:0.942093	train-auc:0.989332
[10]	test-auc:0.942093	train-auc:0.990236
[11]	test-auc:0.945477	train-auc:0.99147
[12]	test-auc:0.94471	train-auc:0.991602
[13]	test-auc:0.951095	train-auc:0.993872
[14]	test-auc:0.958118	train-auc:0.994054
[15]	test-auc:0.963034	train-auc:0.995406
[16]	test-auc:0.962076	train-auc:0.9957
[17]	test-auc:0.963225	train-auc:0.996331
[18]	test-auc:0.964183	train-auc:0.996693
[19]	test-auc:0.969163	train-auc:0.99693
[20]	test-auc:0.972483	train-auc

[49]	test-auc:0.98561	train-auc:0.999998
[50]	test-auc:0.986073	train-auc:0.999998
[51]	test-auc:0.985264	train-auc:0.999996
[52]	test-auc:0.984975	train-auc:0.999999
[53]	test-auc:0.984801	train-auc:0.999998
[54]	test-auc:0.983819	train-auc:0.999999
[55]	test-auc:0.984281	train-auc:0.999999
[56]	test-auc:0.983761	train-auc:1
[57]	test-auc:0.983068	train-auc:1
[58]	test-auc:0.984686	train-auc:1
[59]	test-auc:0.985206	train-auc:1
[60]	test-auc:0.985495	train-auc:1
[61]	test-auc:0.986477	train-auc:1
[62]	test-auc:0.985552	train-auc:1
[63]	test-auc:0.986304	train-auc:1
[64]	test-auc:0.98665	train-auc:1
[65]	test-auc:0.986882	train-auc:1
[66]	test-auc:0.986419	train-auc:1
[67]	test-auc:0.986246	train-auc:1
[68]	test-auc:0.986015	train-auc:1
[69]	test-auc:0.986419	train-auc:1
[70]	test-auc:0.986824	train-auc:1
[71]	test-auc:0.986535	train-auc:1
[72]	test-auc:0.98746	train-auc:1
[73]	test-auc:0.987344	train-auc:1
[74]	test-auc:0.987749	train-auc:1
[75]	test-auc:0.986766	train-auc:1
[76]	test

[109]	test-auc:0.976071	train-auc:1
[110]	test-auc:0.976129	train-auc:1
[111]	test-auc:0.975079	train-auc:1
[112]	test-auc:0.97502	train-auc:1
[113]	test-auc:0.974962	train-auc:1
[114]	test-auc:0.974729	train-auc:1
[115]	test-auc:0.974612	train-auc:1
[116]	test-auc:0.97467	train-auc:1
[117]	test-auc:0.974495	train-auc:1
[118]	test-auc:0.974612	train-auc:1
[119]	test-auc:0.974612	train-auc:1
[120]	test-auc:0.974612	train-auc:1
[121]	test-auc:0.974612	train-auc:1
[122]	test-auc:0.974495	train-auc:1
[123]	test-auc:0.975429	train-auc:1
[124]	test-auc:0.975604	train-auc:1
[125]	test-auc:0.975721	train-auc:1
[126]	test-auc:0.975779	train-auc:1
[127]	test-auc:0.976246	train-auc:1
[128]	test-auc:0.975721	train-auc:1
[129]	test-auc:0.97502	train-auc:1
[130]	test-auc:0.974962	train-auc:1
[131]	test-auc:0.975137	train-auc:1
[132]	test-auc:0.975079	train-auc:1
[133]	test-auc:0.975487	train-auc:1
[134]	test-auc:0.974962	train-auc:1
[135]	test-auc:0.975371	train-auc:1
[136]	test-auc:0.975896	train-a

[20]	test-auc:0.924126	train-auc:0.99914
[21]	test-auc:0.925464	train-auc:0.999301
[22]	test-auc:0.923824	train-auc:0.999435
[23]	test-auc:0.92896	train-auc:0.999483
[24]	test-auc:0.930211	train-auc:0.999589
[25]	test-auc:0.926198	train-auc:0.999702
[26]	test-auc:0.929305	train-auc:0.999793
[27]	test-auc:0.930514	train-auc:0.999864
[28]	test-auc:0.927967	train-auc:0.999893
[29]	test-auc:0.933707	train-auc:0.99992
[30]	test-auc:0.934268	train-auc:0.999942
[31]	test-auc:0.933448	train-auc:0.999951
[32]	test-auc:0.931204	train-auc:0.999962
[33]	test-auc:0.928528	train-auc:0.999971
[34]	test-auc:0.928269	train-auc:0.999982
[35]	test-auc:0.925248	train-auc:0.99998
[36]	test-auc:0.923824	train-auc:0.999989
[37]	test-auc:0.923867	train-auc:0.999989
[38]	test-auc:0.925939	train-auc:0.999993
[39]	test-auc:0.925852	train-auc:0.999997
[40]	test-auc:0.927967	train-auc:0.999997
[41]	test-auc:0.92801	train-auc:0.999997
[42]	test-auc:0.930902	train-auc:0.999998
[43]	test-auc:0.929564	train-auc:0.9999

[88]	test-auc:0.988856	train-auc:1
[89]	test-auc:0.988593	train-auc:1
[90]	test-auc:0.988488	train-auc:1
[91]	test-auc:0.988488	train-auc:1
[92]	test-auc:0.988804	train-auc:1
[93]	test-auc:0.989382	train-auc:1
[94]	test-auc:0.989172	train-auc:1
[95]	test-auc:0.988593	train-auc:1
[96]	test-auc:0.988436	train-auc:1
[97]	test-auc:0.988015	train-auc:1
[98]	test-auc:0.988173	train-auc:1
[99]	test-auc:0.988383	train-auc:1
[100]	test-auc:0.987963	train-auc:1
[101]	test-auc:0.988331	train-auc:1
[102]	test-auc:0.987963	train-auc:1
[103]	test-auc:0.988173	train-auc:1
[104]	test-auc:0.98812	train-auc:1
[105]	test-auc:0.988015	train-auc:1
[106]	test-auc:0.987857	train-auc:1
[107]	test-auc:0.9877	train-auc:1
[108]	test-auc:0.987227	train-auc:1
[109]	test-auc:0.987069	train-auc:1
[110]	test-auc:0.987332	train-auc:1
[111]	test-auc:0.986754	train-auc:1
[112]	test-auc:0.986281	train-auc:1
[113]	test-auc:0.986964	train-auc:1
[114]	test-auc:0.987016	train-auc:1
[115]	test-auc:0.9877	train-auc:1
[116]	tes

...

[89]	test-auc:0.96023	train-auc:1
[90]	test-auc:0.960185	train-auc:1
[91]	test-auc:0.96054	train-auc:1
[92]	test-auc:0.959919	train-auc:1
[93]	test-auc:0.959831	train-auc:1
[94]	test-auc:0.960629	train-auc:1
[95]	test-auc:0.960407	train-auc:1
[96]	test-auc:0.959387	train-auc:1
[97]	test-auc:0.959387	train-auc:1
[98]	test-auc:0.959032	train-auc:1
[99]	test-auc:0.958411	train-auc:1
[100]	test-auc:0.958234	train-auc:1
[101]	test-auc:0.959343	train-auc:1
[102]	test-auc:0.959298	train-auc:1
[103]	test-auc:0.959742	train-auc:1
[104]	test-auc:0.959298	train-auc:1
[105]	test-auc:0.958943	train-auc:1
[106]	test-auc:0.959476	train-auc:1
[107]	test-auc:0.959121	train-auc:1
[108]	test-auc:0.95952	train-auc:1
[109]	test-auc:0.959431	train-auc:1
[110]	test-auc:0.95921	train-auc:1
[111]	test-auc:0.959121	train-auc:1
[112]	test-auc:0.95921	train-auc:1
[113]	test-auc:0.959919	train-auc:1
[114]	test-auc:0.960097	train-auc:1
[115]	test-auc:0.959564	train-auc:1
[116]	test-auc:0.959609	train-auc:1
[117]	te

[18]	test-auc:0.950459	train-auc:0.998775
[19]	test-auc:0.95124	train-auc:0.999002
[20]	test-auc:0.953621	train-auc:0.999071
[21]	test-auc:0.956237	train-auc:0.999442
[22]	test-auc:0.955261	train-auc:0.99963
[23]	test-auc:0.95772	train-auc:0.999709
[24]	test-auc:0.956393	train-auc:0.999775
[25]	test-auc:0.958306	train-auc:0.999833
[26]	test-auc:0.962132	train-auc:0.999891
[27]	test-auc:0.964045	train-auc:0.999941
[28]	test-auc:0.962756	train-auc:0.999973
[29]	test-auc:0.962132	train-auc:0.999979
[30]	test-auc:0.963576	train-auc:0.999974
[31]	test-auc:0.964513	train-auc:0.999983
[32]	test-auc:0.964942	train-auc:0.999987
[33]	test-auc:0.96381	train-auc:0.999995
[34]	test-auc:0.964513	train-auc:0.999998
[35]	test-auc:0.964747	train-auc:0.999998
[36]	test-auc:0.966192	train-auc:0.999999
[37]	test-auc:0.966309	train-auc:1
[38]	test-auc:0.965684	train-auc:1
[39]	test-auc:0.965372	train-auc:1
[40]	test-auc:0.965489	train-auc:0.999999
[41]	test-auc:0.967012	train-auc:1
[42]	test-auc:0.967285	t

[96]	test-auc:0.979435	train-auc:1
[97]	test-auc:0.979337	train-auc:1
[98]	test-auc:0.979045	train-auc:1
[99]	test-auc:0.978996	train-auc:1
[100]	test-auc:0.978801	train-auc:1
[101]	test-auc:0.979435	train-auc:1
[102]	test-auc:0.979678	train-auc:1
[103]	test-auc:0.979776	train-auc:1
[104]	test-auc:0.980117	train-auc:1
[105]	test-auc:0.980361	train-auc:1
[106]	test-auc:0.980312	train-auc:1
[107]	test-auc:0.980556	train-auc:1
[108]	test-auc:0.980556	train-auc:1
[109]	test-auc:0.980848	train-auc:1
[110]	test-auc:0.981092	train-auc:1
[111]	test-auc:0.980897	train-auc:1
[112]	test-auc:0.980556	train-auc:1
[113]	test-auc:0.980214	train-auc:1
[114]	test-auc:0.980361	train-auc:1
[115]	test-auc:0.980409	train-auc:1
[116]	test-auc:0.980799	train-auc:1
[117]	test-auc:0.981384	train-auc:1
[118]	test-auc:0.980945	train-auc:1
[119]	test-auc:0.981287	train-auc:1
[120]	test-auc:0.981092	train-auc:1
[121]	test-auc:0.981335	train-auc:1
[122]	test-auc:0.981189	train-auc:1
[123]	test-auc:0.980653	train-au

[18]	test-auc:0.944635	train-auc:0.998606
[19]	test-auc:0.944731	train-auc:0.998967
[20]	test-auc:0.94454	train-auc:0.999102
[21]	test-auc:0.94096	train-auc:0.999195
[22]	test-auc:0.942965	train-auc:0.999445
[23]	test-auc:0.942679	train-auc:0.999575
[24]	test-auc:0.944206	train-auc:0.999623
[25]	test-auc:0.947213	train-auc:0.999669
[26]	test-auc:0.945256	train-auc:0.999796
[27]	test-auc:0.944874	train-auc:0.999859
[28]	test-auc:0.940769	train-auc:0.999932
[29]	test-auc:0.939719	train-auc:0.999959
[30]	test-auc:0.941819	train-auc:0.999955
[31]	test-auc:0.942058	train-auc:0.999964
[32]	test-auc:0.938192	train-auc:0.999974
[33]	test-auc:0.942631	train-auc:0.999987
[34]	test-auc:0.943872	train-auc:0.999994
[35]	test-auc:0.944158	train-auc:0.999996
[36]	test-auc:0.945924	train-auc:0.999998
[37]	test-auc:0.944969	train-auc:0.999999
[38]	test-auc:0.942344	train-auc:0.999998
[39]	test-auc:0.942297	train-auc:0.999999
[40]	test-auc:0.942058	train-auc:1
[41]	test-auc:0.944635	train-auc:1
[42]	tes

[95]	test-auc:0.947295	train-auc:1
[96]	test-auc:0.947542	train-auc:1
[97]	test-auc:0.947749	train-auc:1
[98]	test-auc:0.947418	train-auc:1
[99]	test-auc:0.946675	train-auc:1
[100]	test-auc:0.945808	train-auc:1
[101]	test-auc:0.946221	train-auc:1
[102]	test-auc:0.946055	train-auc:1
[103]	test-auc:0.945931	train-auc:1
[104]	test-auc:0.946427	train-auc:1
[105]	test-auc:0.946799	train-auc:1
[106]	test-auc:0.946221	train-auc:1
[107]	test-auc:0.946179	train-auc:1
[108]	test-auc:0.945973	train-auc:1
[109]	test-auc:0.946262	train-auc:1
[110]	test-auc:0.946303	train-auc:1
[111]	test-auc:0.946221	train-auc:1
[112]	test-auc:0.946427	train-auc:1
[113]	test-auc:0.946179	train-auc:1
[114]	test-auc:0.946303	train-auc:1
[115]	test-auc:0.945436	train-auc:1
[116]	test-auc:0.945642	train-auc:1
[117]	test-auc:0.944734	train-auc:1
[118]	test-auc:0.945188	train-auc:1
[119]	test-auc:0.945725	train-auc:1
[120]	test-auc:0.945808	train-auc:1
[121]	test-auc:0.945931	train-auc:1
[122]	test-auc:0.946014	train-auc